In [1]:
# importing the needed libraries

import numpy as np
import pandas as pd
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import warnings  
warnings.filterwarnings('ignore')
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold

In [88]:
torch.cuda.is_available()
torch.cuda.get_device_name(0)
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

GeForce MX130
Memory Usage:
Allocated: 3.0 GB
Cached:    3.0 GB


In [89]:
folder_path = "C:/Users/Kaushik/Desktop/anacondaDocs/plant pathology kaggle 2020"

train_df = pd.read_csv(folder_path+"/train.csv")

In [90]:
train_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [91]:
# hyper parameter
SEED = 42
N_FOLDS = 5
N_EPOCHS = 10
BATCH_SIZE = 4
SIZE = 512
input_dir = "C:/Users/Kaushik/Desktop/anacondaDocs/plant pathology kaggle 2020"

## Custom dataset

In [92]:
class plantDataset(Dataset):
    def __init__(self, df, transforms = None):
        self.df = df
        self.transforms = transforms
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        img_src = input_dir + "/images/" + self.df.loc[idx, 'image_id'] + ".jpg"
        image = cv2.imread(img_src, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        labels = self.df.loc[idx,["healthy", "multiple_diseases", "rust", "scab"]].values
        labels = torch.from_numpy(labels.astype(np.int8))
        labels = labels.unsqueeze(-1)
        
        if(self.transforms):
            transformed = self.transforms(image = image)
            image = transformed['image']
        
        return image, labels

In [93]:
class plantModel(nn.Module):
    def __init__(self, num_classes=4):
        super().__init__()
        
        self.backbone = torchvision.models.resnet18(pretrained=True)
        
        in_features = self.backbone.fc.in_features
        
        self.logit = nn.Linear(in_features, num_classes)
        
    def forward(self, x):
        
        batch_size, C, H, W = x.shape
        
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)
        
        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)
        
        x = F.adaptive_avg_pool2d(x,1).reshape(batch_size, -1)
        x = F.dropout(x, 0.25, self.training)
        
        x = self.logit(x)
        
        return x


In [94]:
transforms_train = A.Compose([
    A.RandomResizedCrop(height=SIZE, width=SIZE, p=1.0),
    A.Flip(),
    A.ShiftScaleRotate(rotate_limit=1.0, p=0.8),

    # Pixels
    A.OneOf([
        A.IAAEmboss(p=1.0),
        A.IAASharpen(p=1.0),
        A.Blur(p=1.0),
    ], p=0.5),

    # Affine
    A.OneOf([
        A.ElasticTransform(p=1.0),
        A.IAAPiecewiseAffine(p=1.0)
    ], p=0.5),

    A.Normalize(p=1.0),
    ToTensorV2(p=1.0),
])

transforms_valid = A.Compose([
    A.Resize(height=SIZE, width=SIZE, p=1.0),
    A.Normalize(p=1.0),
    ToTensorV2(p=1.0),
])

In [95]:
submission_df = pd.read_csv(input_dir+'/sample_submission.csv')
submission_df.iloc[:,1:] = 0
submission_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0,0,0,0
1,Test_1,0,0,0,0
2,Test_2,0,0,0,0
3,Test_3,0,0,0,0
4,Test_4,0,0,0,0


In [104]:
test_dataset = plantDataset(df = submission_df, transforms = transforms_valid)
dataloader_test = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=0, shuffle=False)

In [105]:
train_labels = train_df.iloc[:,1:].values

train_y = train_labels[:, 2] + train_labels[:, 3] * 2 + train_labels[:, 1] * 3

In [106]:
train_df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [107]:
folds = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
oof_preds = np.zeros((train_df.shape[0], 4))

In [108]:
model = plantModel(num_classes = 4)

In [109]:
class DenseCrossEntropy(nn.Module):

    def __init__(self):
        super(DenseCrossEntropy, self).__init__()
        
        
    def forward(self, logits, labels):
        logits = logits.float()
        labels = labels.float()
        
        logprobs = F.log_softmax(logits, dim=-1)
        
        loss = -labels * logprobs
        loss = loss.sum(-1)

        return loss.mean()
    

In [110]:

    
def train_one_fold(i_fold, model, criterion, optimizer, dataloader_train, dataloader_valid):
    
    train_fold_results = []

    for epoch in range(N_EPOCHS):

        print('  Epoch {}/{}'.format(epoch + 1, N_EPOCHS))
        print('  ' + ('-' * 20))

        model.train()
        tr_loss = 0

        for step, batch in enumerate(dataloader_train):

            images = batch[0]
            labels = batch[1]

            images = images.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.float)
            
            outputs = model(images)
            loss = criterion(outputs, labels.squeeze(-1))                
            loss.backward()

            tr_loss += loss.item()

            optimizer.step()
            optimizer.zero_grad()

        # Validate
        model.eval()
        val_loss = 0
        val_preds = None
        val_labels = None

        for step, batch in enumerate(dataloader_valid):

            images = batch[0]
            labels = batch[1]

            if val_labels is None:
                val_labels = labels.clone().squeeze(-1)
            else:
                val_labels = torch.cat((val_labels, labels.squeeze(-1)), dim=0)

            images = images.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.float)

            with torch.no_grad():
                outputs = model(images)

                loss = criterion(outputs, labels.squeeze(-1))
                val_loss += loss.item()

                preds = torch.softmax(outputs, dim=1).data.cpu()

                if val_preds is None:
                    val_preds = preds
                else:
                    val_preds = torch.cat((val_preds, preds), dim=0)


        train_fold_results.append({
            'fold': i_fold,
            'epoch': epoch,
            'train_loss': tr_loss / len(dataloader_train),
            'valid_loss': val_loss / len(dataloader_valid),
            'valid_score': roc_auc_score(val_labels, val_preds, average='macro'),
        })

    return val_preds, train_fold_results

In [111]:
submissions = None
train_results = []

for i_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df, train_y)):
    print("Fold {}/{}".format(i_fold + 1, N_FOLDS))

    valid = train_df.iloc[valid_idx]
    valid.reset_index(drop=True, inplace=True)

    train = train_df.iloc[train_idx]
    train.reset_index(drop=True, inplace=True)    

    dataset_train = plantDataset(df=train, transforms=transforms_train)
    dataset_valid = plantDataset(df=valid, transforms=transforms_valid)

    dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, num_workers=0, shuffle=True)
    dataloader_valid = DataLoader(dataset_valid, batch_size=BATCH_SIZE, num_workers=0, shuffle=False)

    device = torch.device("cuda")

    model = plantModel(num_classes=4)
    model.to(device)

    criterion = DenseCrossEntropy()
    plist = [{'params': model.parameters(), 'lr': 5e-5}]
    optimizer = optim.Adam(plist, lr=5e-5)
    
    val_preds, train_fold_results = train_one_fold(i_fold, model, criterion, optimizer, dataloader_train, dataloader_valid)
    
    oof_preds[valid_idx, :] = val_preds.numpy()
    
    train_results = train_results + train_fold_results

    model.eval()
    test_preds = None

    for step, batch in enumerate(dataloader_test):

        images = batch[0]
        images = images.to(device, dtype=torch.float)

        with torch.no_grad():
            outputs = model(images)

            if test_preds is None:
                test_preds = outputs.data.cpu()
            else:
                test_preds = torch.cat((test_preds, outputs.data.cpu()), dim=0)
    
    
    # Save predictions per fold
    submission_df[['healthy', 'multiple_diseases', 'rust', 'scab']] = torch.softmax(test_preds, dim=1)
    submission_df.to_csv('submission_fold_{}.csv'.format(i_fold), index=False)

    # logits avg
    if submissions is None:
        submissions = test_preds / N_FOLDS
    else:
        submissions += test_preds / N_FOLDS

print("5-Folds CV score: {:.4f}".format(roc_auc_score(train_labels, oof_preds, average='macro')))

Fold 1/5
  Epoch 1/10
  --------------------
  Epoch 2/10
  --------------------
  Epoch 3/10
  --------------------
  Epoch 4/10
  --------------------
  Epoch 5/10
  --------------------
  Epoch 6/10
  --------------------
  Epoch 7/10
  --------------------
  Epoch 8/10
  --------------------
  Epoch 9/10
  --------------------
  Epoch 10/10
  --------------------
Fold 2/5
  Epoch 1/10
  --------------------
  Epoch 2/10
  --------------------
  Epoch 3/10
  --------------------
  Epoch 4/10
  --------------------
  Epoch 5/10
  --------------------
  Epoch 6/10
  --------------------
  Epoch 7/10
  --------------------
  Epoch 8/10
  --------------------
  Epoch 9/10
  --------------------
  Epoch 10/10
  --------------------
Fold 3/5
  Epoch 1/10
  --------------------
  Epoch 2/10
  --------------------
  Epoch 3/10
  --------------------
  Epoch 4/10
  --------------------
  Epoch 5/10
  --------------------
  Epoch 6/10
  --------------------
  Epoch 7/10
  --------------------

In [112]:
train_results = pd.DataFrame(train_results)
train_results.head(10)

,fold,epoch,train_loss,valid_loss,valid_score
0,0,0,0.815172,0.332605,0.939007
1,0,1,0.662906,0.277656,0.943137
2,0,2,0.541682,0.254916,0.950485
3,0,3,0.511343,0.286028,0.953318
4,0,4,0.486531,0.234080,0.953603
5,0,5,0.481634,0.259260,0.950003
6,0,6,0.466737,0.224479,0.956162
7,0,7,0.433945,0.216818,0.959835
8,0,8,0.375031,0.245548,0.955395
9,0,9,0.400806,0.226154,0.957994


In [113]:
submission_df[['healthy', 'multiple_diseases', 'rust', 'scab']] = torch.softmax(submissions, dim=1)
submission_df.to_csv('submission.csv', index=False)

In [114]:
submission_df

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,1.719828e-03,0.022619,0.975380,0.000281
1,Test_1,8.232382e-03,0.070055,0.919165,0.002548
2,Test_2,4.991230e-04,0.002019,0.002002,0.995479
3,Test_3,9.736843e-01,0.000574,0.024189,0.001553
4,Test_4,8.019173e-07,0.000257,0.999704,0.000038
...,...,...,...,...,...
1816,Test_1816,2.071851e-04,0.002230,0.996629,0.000934
1817,Test_1817,2.474055e-01,0.374772,0.120024,0.257798
1818,Test_1818,2.192626e-03,0.036639,0.960907,0.000261
1819,Test_1819,9.965084e-01,0.000977,0.001248,0.001266


In [115]:
 torch.save(model.state_dict(),"trained_model_weights")

In [116]:
model.load_state_dict(torch.load('trained_model_weights'))

<All keys matched successfully>